## Final project

** James N. Vance **

Find the solution of 

$$
-\Delta \mathbf u + \mathbf u = \mathbf f 
$$

on a cube $[0,1]^3$. For the sake of simplicity you can consider just **one finite element** and homogeneous Neumann boundary conditions. 

Consider Lagrangian finite elements, with Chebyshev nodes, of high order (up to 30--40).

The above problem can be rewritten in matrix form as

$$
(K+M)\mathbf u = \mathbf f
$$

where $K$ is the stiffness matrix, and $M$ is the mass matrix.



## 1.
Assemble the matrices and solve the system with a **direct solver**. Plot (loglog) the $L_2$ norm of the error with respect to a known exact solution as function of the degree of the finite element. 

**Solution:**

Our goal is to find the solution to the PDE $-\nabla^2 u + u = f$ 
on the cubic domain $\Omega=[0,1]^3$ with the homogeneous Neumann boundary conditions $\partial u/\partial n = 0$ on $\partial \Omega$. Integrating the PDE over the domain $\Omega$ and imposing the boundary conditions yields the weak formulation
$$
\int_\Omega \nabla u \cdot \nabla v + \int_\Omega u \cdot v = \int_\Omega f \cdot v, ~~~~ \forall v \in H_1^1(\Omega)
$$

We approximate the solution in the finite element subspace
$$
\int_\Omega \nabla u_h \cdot \nabla v_h + \int_\Omega u_h \cdot v_h = \int_\Omega f \cdot v_h, ~~~~ \forall v_h \in V_h
$$
and choose basis functions $\phi_I$ such that
$$u_h = \sum_{I \in \mathcal I_s} c_I \phi_I$$
where $c_I$'s are the degrees of freedom and $\mathcal I_s$ is the index set for the linear expansion.

With the expansion linear expansion proposed earlier, we rewrite the problem into
\begin{align}
\int_\Omega \nabla \left( \sum_{I \in \mathcal I_s } c_I \phi_I  \right) \cdot \nabla  \phi_J + \int_\Omega \left( \sum_{I \in \mathcal I_s } c_I \phi_I  \right) \cdot \phi_J &= \int_\Omega f \cdot \phi_J, ~~~~ \forall J \in \mathcal I_s \\
\sum_{J \in \mathcal I_s } c_J \left( \int_\Omega \nabla \phi_J   \cdot \nabla  \phi_J + \int_\Omega \phi_I \cdot \phi_J \right) &= \int_\Omega f \cdot \phi_J, ~~~~ \forall J \in \mathcal I_s
\end{align}

This is equivalent to solving the linear system
$$
(\hat K + \hat M)\mathbf c = \mathbf F
$$
where
\begin{align}
K_{I,J} &= \int_\Omega \nabla \phi_I   \cdot \nabla  \phi_J, & M_{I,J} &= \int_\Omega \phi_I \cdot \phi_J, & F_{J} &= \int_\Omega f \cdot \phi_J
\end{align}

Using the index notation $I := (i,j,k) \in \mathcal I_s$, we adapt the basis functions in the polynomial space

$$
\mathscr P^{n,m,l}: \mathrm {span}\{ p_{i,j,k}(x,y,z) \}_{i,j,k=0}^{n,m,l} $$

Imposing separability, we may write these polynomials as 

\begin{align}
p_{i,j,k}(x,y,z) = p_i(x) p_j(y) p_k(z)
\end{align}

where $i=0 \dots n,~j=0 \dots m,~k=0 \dots l$, and $n$, $m$ and $l$ are the degrees of the polynomial in $x$,$y$ and $z$, respectively. 

Four each basis, we choose the Lagrange interpolating polynomial, whose form in one dimension is given by

$$
p_i^n(x) = \prod_{j=0,j \neq i}^{n} \frac{x-x_j}{x_i-x_j}
$$

where $x_i$'s are interpolation points corresponding to the nodes of our finite element.

We construct these one-dimensional basis functions given the node points and the degree of interpolation using the following functions:

In [ ]:
import numpy as np
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt


def ConstructDualBasis(nodes): 
    N = []
    for i in range(len(nodes)):
        nodefunc = lambda f, i = i : f(nodes[i])
        N.append(nodefunc)
    return N


def BasisPolynomials(deg):
    P = []
    for i in range(deg+1):
        c = np.zeros((i+1,))
        c[-1] = 1
        P.append(poly.Polynomial(c))
    return P


def ConstructCij(N,P):
    C = np.zeros((0,len(P)))
    for dof in N:
        row = np.array([])
        for p in P:
            row = np.hstack((row,dof(p)))
        C = np.vstack((C,row))
    return C


def ConstructCanonicalBasis(C):
    from scipy.linalg import solve
    
    V = []
    for k in range(C.shape[0]):
        delta_ij = np.zeros(C.shape[0])
        delta_ij[k] = 1
        vk = solve(C,delta_ij)
        V.append(poly.Polynomial(vk))
    return V


To minimize the effect of oscillations at the edges of the interpolation, known as Runge's phenomenon, we use  Chebyshev nodes in determining interpolation points. On the interval $[0,1]$ the Chebyshev nodes are given by

$$ x_k = \frac{1}{2} + \frac{1}{2} \cos\left(\frac{2k-1}{2n}\pi\right), ~~ k=1,2,\dots n $$

In [ ]:
def ChebyshevNodes(n): # n is the number of desired nodes
    return [0.5+0.5*np.cos((2.*k-1.)/(2.*n)*np.pi) for k in range(1,n+1)]


The problem in 3D can be simplified by expressing everything in terms of the similar problem in one dimension

$$
-\partial^2_x u(x) + u(x) = f(x)
$$

whose weak form in the finite element subspace is expressed as 

$$
\int_{0}^{1} u_h'(x) v_h'(x)~\mathrm dx + \int_{0}^{1} u_h(x) v_h(x) ~\mathrm dx  = \int_{0}^{1} f v_h(x), ~~~~ \forall v_h \in V_h
$$

By choosing the expansion,

$$ u_h(x) = \sum_{i=0}^{n} c_i p_i(x)$$

it can be shown that the problem reduces to the linear system

$$ (\hat k + \hat  m) \mathbf c = \mathbf f $$

where 

\begin{align}
k_{i,j} &= \int_{0}^{1}  \phi_i'(x)   \phi_j'(x) \mathrm d x, & m_{i,j} &= \int_{0}^{1}  \phi_i(x)   \phi_j(x) \mathrm d x, & f_{j} &= \int_{0}^{1}  f(x)  \phi_j(x) \mathrm d x
\end{align}




<!--A_{I,J} =& \int_\Omega \nabla (p_i(x) p_j(y)& p_k(z))   \cdot \nabla  (p_q(x) p_r(y) p_s(z)) \\-->

We construct stiffness matrix $\hat k$, mass matrix $\hat m$ and the right-hand side vector $\mathbf f$ for the one dimensional case using the following functions:

In [ ]:
from numpy.polynomial.legendre import leggauss

def ConstructKM1D(degree):
    nnodes = degree + 1
    nodes  = ChebyshevNodes(nnodes)

    N = ConstructDualBasis(nodes) 
    E = BasisPolynomials(degree)
    C = ConstructCij(N,E)
    V = ConstructCanonicalBasis(C)

    nquad = 2*len(N)-1         # number of quadrature points
    q,w   = leggauss(nquad)    # on interval [-1,+1]
    q     = (q+1)/2            # rescale to  [ 0,+1]
    nder  = 2                  # obtain 0th and 1st derivatives

    Lq = np.zeros((nder,nnodes,nquad))
    for i in range(nder):
        for j in range(nnodes):
            Lq[i][j] = V[j].deriv(i)(q)

    M = np.einsum('jq, iq, q -> ij', Lq[0], Lq[0], w)
    K = np.einsum('jq, iq, q -> ij', Lq[1], Lq[1], w)

    return K, M, V


def ConstructF1D(f,V,degree):
    
    nnodes = len(V)
    nquad  = 2*(degree+1)-1     # number of quadrature points
    q,w    = leggauss(nquad)    # on interval [-1,+1]
    q      = (q+1)/2            # rescale to  [ 0,+1]
    nder   = 1                  # obtain 0th and 1st derivatives

    Lq = np.zeros((nder,nnodes,nquad))
    for i in range(nder):
        for j in range(nnodes):
            Lq[i][j] = V[j].deriv(i)(q)        
    
    fq = f(q)
    F = np.einsum('iq, q, q -> i', Lq[0], fq, w)
    
    return F

From the components of the 3D linear system,
\begin{align}
K_{I,J} &= \int_\Omega \nabla \phi_I   \cdot \nabla  \phi_J, & M_{I,J} &= \int_\Omega \phi_I \cdot \phi_J, & F_{J} &= \int_\Omega f \cdot \phi_J
\end{align}
setting 
$$ \phi_I = p_{i,j,k}(x,y,z) = p_i(x) p_j(y) p_k(z) $$ 
and 
$$ \phi_J = p_{q,r,s}(x,y,z) = p_q(x) p_r(y) p_s(z) $$ 

and imposing a separable right-hand side function for simplicity, i.e.

$$
f(x,y,z) = f(x)f(y)f(z)
$$

we can rewrite the the 3D equations as 

\begin{eqnarray}
M_{I,J} =& \int_0^1 p_i(x)p_q(x)~\mathrm dx &
             \int_0^1 p_j(y)p_r(y)~\mathrm dy 
             \int_0^1 p_k(z)p_s(z)~\mathrm dz \\
K_{I,J} =& \int_0^1\int_0^1\int_0^1 \big( &
            p_i'(x) p_j(y) p_k(z) p_q'(x) p_r(y) p_s(z) +p_i(x) p_j'(y) p_k(z) p_q(x) p_r'(y) p_s(z) \\
         &&+p_i(x) p_j(y) p_k'(z) p_q(x) p_r(y) p_s'(z) 
        ~\big)~\mathrm dx~\mathrm dy~\mathrm dz \\
F_{J} =& \int_0^1 f(x)p_q(x)~\mathrm dx &
             \int_0^1 f(y)p_r(y)~\mathrm dy 
             \int_0^1 f(z)p_s(z)~\mathrm dz &
\end{eqnarray}

Thus, the 3D case may now be expressed as tensor products of their one-dimensional equivalents:

\begin{eqnarray}
M_{I,J} &=& m_{i,q} m_{j,r} m_{k,s} \\
K_{I,J} &=& k_{i,q} m_{j,r} m_{k,s} +  m_{i,q} k_{j,r} m_{k,s} + m_{i,q} k_{j,r} a_{k,s} \\
F_{J} &=& f_{q} f_{r} f_{s}
\end{eqnarray}

The following function assembles the 3D linear system and calls a function `linsolve` to calculate the solution

In [ ]:
def FEM_solve(degree,f_rhs,solver=None,view=False,P=None):
    k,m,v = ConstructKM1D(degree)
    f     = ConstructF1D(f_rhs,v,30)               # Degree of RHS independent of LHS

    F = np.einsum('i,j,k->ijk',f,f,f)              # Right-hand side
    F = np.reshape(F,np.prod(F.shape))

    if view:                                       # Also returns the full linear system
        M  = np.einsum('ij,kl,mn->ikmjln',m,m,m)   # Mass matrix
        M  = np.reshape(M, (np.prod(M.shape[:3]),np.prod(M.shape[3:])))
        K  = np.einsum('ij,kl,mn->ikmjln',k,m,m)   # Stiffness matrix
        K += np.einsum('ij,kl,mn->ikmjln',m,k,m)
        K += np.einsum('ij,kl,mn->ikmjln',m,m,k)
        K  = np.reshape(K, (np.prod(K.shape[:3]),np.prod(K.shape[3:])))

        c = linsolve(K+M,F,solver=solver,overwrite=False,P=P)

        return c,v,K,M,F

    else:                                          # Returns only the result
        A  = np.einsum('ij,kl,mn->ikmjln',m,m,m)       
        A += np.einsum('ij,kl,mn->ikmjln',k,m,m)
        A += np.einsum('ij,kl,mn->ikmjln',m,k,m)
        A += np.einsum('ij,kl,mn->ikmjln',m,m,k)
        A  = np.reshape(A, (np.prod(A.shape[:3]),np.prod(A.shape[3:])))

        c = linsolve(A,F,solver=solver,overwrite=True,P=P)

        return c,v
    

The linear solver that we have implemented uses the Cholesky decomposition approach since the matrix $\hat A = \hat K + \hat M$ is symmetric and positive definite. We also decorated the function with the numba `@jit` directive for increased performance. 

In [ ]:
from numba import jit

@jit(nopython=True)
def cholesky(A):
    N = len(A)
    for k in range(N-1):
        A[k,k] = np.sqrt(A[k,k])
        A[k+1:N,k] = A[k+1:N,k]/A[k,k]
        for j in range(k+1,N):
            A[j:N,j] = A[j:N,j] - A[j:N,k]*A[j,k]     
    A[-1,-1] = np.sqrt(A[-1,-1])
    return A

@jit(nopython=True)
def L_solve(L,rhs):
    N = len(L)
    sol = np.zeros(N)
    sol[0] = rhs[0]/L[0,0]
    for k in range(1,N):
        sol[k] = (rhs[k] - np.dot(L[k,:k], sol[:k])) / L[k,k]        
    return sol

@jit(nopython=True)
def U_solve(U,rhs):
    N = len(U)
    sol = np.zeros(N)
    sol[-1] = rhs[-1]/U[-1,-1]
    for k in range(N-1):
        sol[N-k-2] = (rhs[N-k-2] - np.dot(U[N-k-2,N-k-1:N-1], sol[N-k-1:N-1])) / U[N-k-2,N-k-2]
    return sol


The generic function `linsolve` allows us to choose which type of solver to use.

In [ ]:
def linsolve(A,b,solver=None,overwrite=False,P=None):
    if solver == 'my cholesky':
        if overwrite:
            A1 = A
        else:
            A1 = A.copy()
        HT = cholesky(A1)
        HT =np.tril(HT)
        H = HT.transpose()
        y = L_solve(HT,b)
        c = U_solve(H,y)
        
    elif solver == 'my cg':
        c = linsolve_my_cg(A,b,overwrite,P)
        # implementation of iterative solver as provided in section 2
        
    elif solver == 'scipy cholesky':
        from scipy.linalg import cho_factor, cho_solve
        c,lower = cho_factor(A,b,overwrite_a=overwrite)
        c = cho_solve((c, lower),b,overwrite_b=overwrite)
        
    elif solver == 'scipy cg':
        from scipy.sparse.linalg import cg
        c,info = cg(A,b) 
        
    elif solver == 'scipy linalg':
        from scipy.linalg import solve
        c = solve(A,b)
        
    else:
        raise NameError, "Solver not specified"
    
    return c

We also asses the L2 norm of the error using the following function with uniform mesh

In [ ]:
def L2norm(U): 
    return np.sqrt(np.sum(U*U))/(np.product(U.shape))


We also interpolate the solution to a uniform mesh for plotting and error calculation:

In [ ]:
def FEM_error(degree,u_ex,c,v,ns,view=False):
    n = degree + 1
    s = np.linspace(0,1,ns)
    Ls = np.zeros((n, ns))
    for j in range(n):
        Ls[j] = v[j](s)
    C = np.einsum('is, jk, pq -> skqijp', Ls, Ls, Ls)

    uex1D = u_ex(s)
    Uex   = np.einsum('i,j,k->ijk',uex1D,uex1D,uex1D)

    U = np.einsum('skqijp, ijp', C, c.reshape((n,n,n)))
    
    if view:
        return L2norm(U-Uex),U,Uex,s
    else:
        return L2norm(U-Uex)
    

And we write some miscellaneous for plotting:

In [ ]:
def surfplot(X,Y,U,filename=''):
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib import cm
    import matplotlib.pyplot as plt
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, U, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    if filename != '': plt.savefig(filename)
    plt.show()    


def cmplot(U,filename=''):
    import matplotlib.pyplot as plt
    plt.imshow(U)
    plt.colorbar()
    if filename != '': plt.savefig(filename)
    plt.show()
    

Now, we execute a sample calculation using the exact solution:

$$
u_{\mathrm{ex}}(x,y,z) = \cos(kx)\cos(ky)\cos(kz)
$$

which satisfies the boundary conditions $\partial u/\partial n = 0$ at the boundaries. The right-hand side becomes

$$
f(x,y,z) = (1+k^2)^3 \cos(kx)\cos(ky)\cos(kz)
$$




In [ ]:
kk     = 3*np.pi
f_rhs  = lambda x: np.power(1+kk**2,1.0/3.0)*np.cos(kk*x)
u_ex   = lambda x: np.cos(kk*x) 

deg = 10
ns  = 30

c,v,K,M,F       = FEM_solve(deg,f_rhs,'my cholesky',True)
err_fem,U,Uex,s = FEM_error(deg,u_ex,c,v,deg+1,True) # Interpolated with same degree as FE
err_int,U,Uex,s = FEM_error(deg,u_ex,c,v,ns,True)    # Interpolated with higher degree than FE

The following plots the solution $u(x,y,z)$ at $z=2.5$. Also shown is a plot of the corresponding matrix values.

In [ ]:
U_slice = U[int(len(s)/4)]
X, Y = np.meshgrid(s,s)
surfplot(X,Y,U_slice)
cmplot(U_slice)
print("FEM Error: {}".format(err_fem))

We now look at the relationship between the degree of the finite element with the $L_2$ norm of the error using the following function.

In [ ]:
def FEM_assess(deg0,deg1,solver,P=None):
    from time import time

    kk     = 3*np.pi
    f_rhs  = lambda x: np.power(1+kk**2,1.0/3.0)*np.cos(kk*x)
    u_ex   = lambda x: np.cos(kk*x)

    degrees  = range(deg0,deg1+1)
    error    = []
    runtime  = []
    
    filename = "data_{}_{}_{}_{}_.txt".format(solver,str(P),degrees[0],degrees[-1])
    print("Writing data to {}".format(filename))
    print("Degree\tL2norm of error\tTime".format(filename))
    with open(filename,"w") as f:
        f.write('')

    loop_start = time()
    for degree in degrees:
        start = time()
        c,v = FEM_solve(degree,f_rhs,solver,False,P=P)
        error.append(FEM_error(degree,u_ex,c,v,len(v)))
        runtime.append(time()-start)
        line = "%d\t%f\t%f"%(degree, error[-1], runtime[-1])
        with open(filename,"a") as f: 
            f.write(line)
            f.write("\n")
        print(line)
    print("Loop:\t\t\t%f"%(time()-loop_start))

    return degrees, error, runtime

def errorplot(degrees, error):
    plt.loglog(degrees,error,'o-')
    plt.xlabel('Degree of finite element')
    plt.ylabel('$L_2$ norm of error')
    plt.show()

def timeplot(degrees, runtime):
    plt.semilogy(degrees,runtime,'o-')
    plt.xlabel('Degree of finite element')
    plt.ylabel('Runtime (s)')
    plt.show()

**Note:** To speed up calculation, `scipy cholesky` is used by default. Custom solver `my cholesky` may be specified instead.

In [ ]:
degrees_ch_loc, error_ch_loc, runtime_ch_loc = FEM_assess(3,12,'my cholesky') 

In [ ]:
errorplot(degrees_ch_loc, error_ch_loc)

In [ ]:
timeplot(degrees_ch_loc, runtime_ch_loc)

We also ran the calculations on the C3E cluster and obtained the following results:

In [ ]:
degrees_ch_c3e, error_ch_c3e, runtime_ch_c3e = np.genfromtxt("./project/data_chol.txt").T

In [ ]:
errorplot(degrees_ch_c3e, error_ch_c3e)

In [ ]:
timeplot(degrees_ch_c3e, runtime_ch_c3e)

## 2.
Use an iterative solver with a chosen preconditioner. Compare and comment the computational burden between the direct and iterative case.

We choose the conjugate gradient method as our iterative solver since the LHS matrix $\hat A = \hat K + \hat M$ is symmetric and positive definite. The function `CG` provides the implementation with an optional preconditioner `P`. As preconditioner, we implement the Incomplete Cholesky decomposition

The function `linsolve_my_cg` is called from the earlier implemented `linsolve` to perform the conjugate gradient method.

In [ ]:
from scipy.linalg import norm,solve

In [ ]:
from numpy.linalg import norm,solve


def CG(A,b,P=None,eps=1e-10):
    
    x = np.zeros_like(b)    
    res = b - np.dot(A,x)

    if P is not None:
        z = solve(P,res)
    else:
        z = res.copy() 
    
    p = z.copy()

    x = CG_loop(A,P,p,x,res,eps)
                 
    return x


@jit(nopython=True)
def CG_loop(A,P,p,x,res,eps):
    it = 0
    n = len(A)
    tol = eps + 1
    while (it < n and tol > eps):
        it    += 1
        q      = np.dot(A,p)
        pAp    = np.dot(p,q)                
        alpha  = np.dot(p,res) / pAp
        x     += alpha * p
        res   -= alpha * q
        tol    = norm(res,2)
        
        if P is not None:
            z = solve(P,res)
        else:
            z = res.copy()
        
        beta = np.dot(q,z) / pAp
        p    = z - beta * p   
    
    return x


def linsolve_my_cg(A,b,overwrite,P):
    if P is None:
        c = CG(A,b)
    elif P == 'A':
        c = CG(A,b,A)
    elif P == 'jacobi':
        pre = np.diag(np.diag(A))
        c   = CG(A,b,pre)
    elif P == 'incomplete cholesky':
        pre = A.copy()
        pre = IncompleteCholesky(pre)
        c   = CG(A,b,pre)
    else:
        print("Specify correct preconditioner")
        return None        
    return c


@jit(nopython=True)
def IncompleteCholesky(P):
    n = len(P)

    for k in range(n):
        P[k,k] = np.sqrt(P[k,k])
        for i in range(k+1,n):
            if (P[i,k]!=0):
                P[i,k] = P[i,k]/P[k,k]
        for j in range(k+1,n):
            for i in range(j,n):
                if P[i,j] != 0:
                    P[i,j] = P[i,j]-P[i,k]*P[j,k]
        
    for i in range(n):
        for j in range(i+1,n):
            P[i,j] = 0
    
    return P
        

In [ ]:
kk     = 3*np.pi
f_rhs  = lambda x: np.power(1+kk**2,1.0/3.0)*np.cos(kk*x)
u_ex   = lambda x: np.cos(kk*x) 

deg = 10
ns  = 30

c,v,K,M,F       = FEM_solve(deg,f_rhs,'my cg',True,P='incomplete cholesky')
err_fem,U,Uex,s = FEM_error(deg,u_ex,c,v,deg+1,True) # Interpolated with same degree as FE
err_int,U,Uex,s = FEM_error(deg,u_ex,c,v,ns,True)    # Interpolated with higher degree than FE

In [ ]:
U_slice = U[int(len(s)/3)]
X, Y = np.meshgrid(s,s)
surfplot(X,Y,U_slice)
cmplot(U_slice)
print("FEM Error: {}".format(err_fem))

In [ ]:
degrees_cg_loc, error_cg_loc, runtime_cg_loc = FEM_assess(3,12,'my cg','jacobi')

In [ ]:
errorplot(degrees_cg_loc,error_cg_loc)

In [ ]:
timeplot(degrees_cg_loc,runtime_cg_loc)

In [ ]:
degrees_cg_c3e, error_cg_c3e, runtime_cg_c3e = np.genfromtxt("./project/data_cg.txt").T

In [ ]:
plt.loglog(degrees_ch_c3e,error_ch_c3e,'x-')
errorplot(degrees_cg_c3e,error_cg_c3e)

In [ ]:
plt.loglog(degrees_ch_c3e,runtime_ch_c3e,'x-') # turn off jit for ch
timeplot(degrees_cg_c3e,runtime_cg_c3e)

### Remarks:

Compare and comment on the computational burden

3 - **OPTIONAL** Using the iterative solver, implement a matrix-free approach where the computation of the matrices is substituted with a  function which returns the *matrix-vector* product. In doing so, you must take care of rewriting the assemble of the matrix as a sum of many *matrix-matrix* products exploiting all the possible tensor products. 

Compare and comment the compuational cost of 100 matrix-vector product when the matrix is assembled and when it is computed on-the-fly as a function of the degree of the finite element.